# Raport z zadania sztuczne sieci neuronowe

In [1]:
from preprocessing import MnistDataloader
from experiment import experiment, plot_costs
from model import Model
from layer import Layer
import numpy as np
from PIL import Image as im

In [2]:
mnist_dataloader = MnistDataloader("train-images-idx3-ubyte", "train-labels-idx1-ubyte", "t10k-images.idx3-ubyte", "t10k-labels.idx1-ubyte")
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

In [3]:
def view_image(image):
    image = np.array(image)
    data = im.fromarray(image)
    data.save("test.png")

In [4]:
model = Model()
model.add_module(Layer((784, 256), "relu"))
model.add_module(Layer((256, 128), "relu"))
model.add_module(Layer((128, 64), "relu"))
model.add_module(Layer((64, 10), "softmax"))


In [5]:
def train(X, y, epochs, learning_rate, batch_size):
    dataset_size = len(X)
    for epoch in range(epochs):
        # print(f"Epoch: {epoch}")
        for i in range(0,dataset_size,batch_size):
            # print(f"batch: {i}")
            batch_X = np.array(X[i:i+batch_size]) / 255.0
            batch_Y = np.array(y[i:i+batch_size])
            loss = model.step(batch_X, batch_Y, learning_rate)
        if epoch % 10 == 0:
            print(f"Epoch: {epoch}, loss: {loss}")

In [ ]:
def predict(X, label):
    pred = model.predict(np.array(X))
    return f"Prediction: {pred}, Label: {label}"

In [ ]:
train(x_train, y_train, 100, 3e-4, 128)

Epoch: 0, loss: 0.03843716743479128
Epoch: 10, loss: 0.02275984614788316
Epoch: 20, loss: 0.02079451024686997
Epoch: 30, loss: 0.01828783820998918
Epoch: 40, loss: 0.01045114631347432
Epoch: 50, loss: 0.0013841048134248428
Epoch: 60, loss: 0.0004083700946705055


In [9]:
print(predict(x_test))

(10000, 10)
[0 0 0 ... 0 0 0]


In [20]:
import torch.nn as nn
import torch.nn.functional as F
import torch
device = torch.device("cuda"if torch.cuda.is_available() else "cpu")

In [31]:
class TorchModel(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.linear1 = nn.Linear(in_features=784, out_features=256)
        self.linear2 = nn.Linear(in_features=256, out_features=128)
        self.linear3 = nn.Linear(in_features=128, out_features=64)
        self.linear4 = nn.Linear(in_features=64, out_features=32)
        if device.type == "cuda":
            self.cuda()

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = self.linear4(x)
        return x

In [32]:
class MnistDataset(torch.utils.data.Dataset):
    def __init__(self, annotations_list, img_array, transform=None, target_transform=None):
        self.img_labels = annotations_list
        self.images = img_array
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx], dtype=torch.float32)
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
def torch_train(dataset, model, learning_rate, epochs, batch_size):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    optimizer = torch.optim.SGD(model.parameters(), learning_rate)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(epochs):
        for image, label in dataloader:
            optimizer.zero_grad()
            y_pred = model((image/ 255.0).to(device))
            loss = criterion(y_pred, label.to(device))
            print(f"Loss:{loss}")
            loss.backward()
            optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch:{epoch}, Loss: {loss}")

In [34]:
dataset = MnistDataset(np.array(y_train), np.array(x_train))
torch_model = TorchModel(device)

In [35]:
torch_train(dataset, torch_model, 3e-4, 100, 128)

Loss:7.183226585388184
Loss:5.526722431182861
Loss:5.632143020629883
Loss:4.806131362915039
Loss:4.688581943511963
Loss:4.7320780754089355
Loss:4.388777732849121
Loss:4.280552387237549
Loss:3.8892598152160645
Loss:4.133275985717773
Loss:3.5533838272094727
Loss:3.6376161575317383
Loss:3.6535890102386475
Loss:3.619192123413086
Loss:3.247619390487671
Loss:3.2427988052368164
Loss:3.1567389965057373
Loss:3.2508623600006104
Loss:3.12253999710083
Loss:3.07212233543396
Loss:2.838172197341919
Loss:2.819213628768921
Loss:2.9339609146118164
Loss:2.758969783782959
Loss:2.686230182647705
Loss:2.6710498332977295
Loss:2.5008907318115234
Loss:2.6712229251861572
Loss:2.5591840744018555
Loss:2.8358194828033447
Loss:2.4263806343078613
Loss:2.398364305496216
Loss:2.4989407062530518
Loss:2.1339433193206787
Loss:2.5319674015045166
Loss:2.1941096782684326
Loss:2.3115954399108887
Loss:2.4435923099517822
Loss:2.187356472015381
Loss:2.3480963706970215
Loss:2.1209006309509277
Loss:2.0830483436584473
Loss:1.91945

KeyboardInterrupt: 

In [11]:
def predict(x, model):
    y_pred = model(x.to(device))
    return y_pred

In [22]:
print(predict(torch.tensor(x_test[0], dtype=torch.float32), torch_model))

torch.Size([28, 28])
xd torch.Size([256, 784])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (28x28 and 784x256)

# Wyniki
- Najlepszą dokładność na zbiorze walidacyjnym osiągnął model trzeci, z jedną ukrytą warstwą o rozmiarze 20, learning ratem o wartości 0.006 oraz batch sizem równym 128.
- W modelu numer cztery w trakcie treningu doszło do przepełnienia.
- Pozostałe modele osiągnęły dokładność na zbiorze walidacyjnym w granicach 80-85%.
- Model trzeci na zbiorze testowym osiągnął wynik 93%.
# Wnioski
- Modele z wiekszą ilością ukrytych warstw oraz ukrytych neuronów nie zawsze osiągały wyższą dokładność. Architektura modelu powinna być ściśle dopasowana do poziomu złożoności danych - nie może być za prosta (jak w modelu 1) ani zbyt rozbudowana (jak w modelu 0).
- Częste aktualizowanie parametrów w przypadku małego batch sizu może mieć negatywny wpływ na dokładność.
- Mniejszy learning rate w modelu o bardziej rozbudowanej architekturze zapobiegał "eksplozji" gradientu.
- Dokładność treningowa nie odbiegała zbytnio od dokładności walidacyjnej (największa różnica w modelu trzecim). Nie doszło do zjawiska zbytniego overfittingu.